In [15]:
#get image size info and bbox index info
import glob
import numpy as np
import os
import pandas as pd

In [2]:
def get_npy_shapes_recursive_with_glob(root_dir):
    npy_shapes = []
    pattern = os.path.join(root_dir, '**', '*.npy')  # Recursive glob pattern
    for file_path in glob.glob(pattern, recursive=True):
        try:
            data = np.load(file_path, mmap_mode='r')
            if isinstance(data, np.ndarray) and data.ndim == 2:
                npy_shapes.append((file_path, data.shape))
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    return npy_shapes

In [ ]:
root_directory = "/data1/lidc-idri/slices"
shapes = get_npy_shapes_recursive_with_glob(root_directory)
for path, shape in shapes:
    print(f"{path}: shape = {shape}")


/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_050_4.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_047_2.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_048_4.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_051_4.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_054_3.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_049_4.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0434/slice_053_3.npy: shape = (360, 360)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_073_3.npy: shape = (326, 326)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_059_1.npy: shape = (326, 326)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_060_1.npy: shape = (326, 326)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_074_3.npy: shape = (326, 326)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_072_3.npy: shape = (326, 326)
/data1/lidc-idri/slices/LIDC-IDRI-0354/slice_061_1.npy: shape = (326, 326)
/data1/lidc-idri/slices/L

In [8]:
sizes = np.array([shape[1] for shape in shapes])

In [16]:
pd.DataFrame(sizes,columns=['xsize','ysize']).describe()

,xsize,ysize
count,7849.000000,7849.000000
mean,356.106510,356.106510
std,43.645574,43.645574
min,236.000000,236.000000
25%,329.000000,329.000000
50%,360.000000,360.000000
75%,380.000000,380.000000
max,500.000000,500.000000


In [ ]:
import matplotlib.pyplot as plt
plt.hist(sizes[:,0])

In [ ]:
bbdf = pd.read_json("/data2/lijin/lidc-prep/kjs/bbox/bbdata.json")

320

In [ ]:
bbdf